In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/question-pairs-dataset/questions.csv
/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin


In [2]:
df=pd.read_csv("/kaggle/input/question-pairs-dataset/questions.csv")

In [3]:
len(df)

404351

In [4]:
df_test=df[300000:350000]

In [5]:
df_test=df_test.astype(str)

In [6]:
df_test

,id,qid1,qid2,question1,question2,is_duplicate
300000,300000,589215,589216,How do I prepare for interviews for cse?,What is the best way to prepare for cse?,0
300001,300001,589217,589218,What is the best bicycle to buy under 10k?,Which is the best bike in in dia to buy in INR...,1
300002,300002,589219,589220,How do I become Mutual funds distributer for a...,How do I become mutual funds distributor for a...,1
300003,300003,589221,589222,Will this relationship work?,Relationship: Will this relationship work?,0
300004,300004,589223,589224,How does Brexit affect India?,Will the GBP/AUD be affected by Brexit?,0
...,...,...,...,...,...,...
349995,349995,685673,685674,How do the automated cars work?,How do automated cars work?,1
349996,349996,685675,685676,Would quantam computers be more powerful or ad...,Why doesn't NFL street 3 play on my PS2 slim?,0
349997,349997,685677,685678,Why did you choose hotel management?,Why does you chosen hotel management?,1
349998,349998,685679,685680,If energy is not conserved in an expanding uni...,Is dark energy and vacuum energy potential ene...,1


# As the dataset is very big we will take only 2 lakhs rows...

In [7]:
df=pd.read_csv("/kaggle/input/question-pairs-dataset/questions.csv",nrows=200000).astype(str)
df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [8]:
df.count()

id              200000
qid1            200000
qid2            200000
question1       200000
question2       200000
is_duplicate    200000
dtype: int64

In [9]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [10]:
df['is_duplicate'].nunique()

2

In [11]:
df.describe()

,id,qid1,qid2,question1,question2,is_duplicate
count,200000,200000,200000,200000,200000,200000
unique,200000,198651,198254,160509,162390,2
top,28093,7863,65333,How do I improve my pronunciation of English?,How can you look at someone's private Instagra...,0
freq,1,17,20,25,54,125519


In [12]:
df.dtypes

id              object
qid1            object
qid2            object
question1       object
question2       object
is_duplicate    object
dtype: object

In [13]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding,Activation,Flatten,merge,TimeDistributed,Bidirectional
from keras.layers.merge import concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import concatenate,subtract,add,maximum,multiply,Layer,Lambda
from keras.backend import backend as K

Using TensorFlow backend.


In [14]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import nltk 
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [15]:
import re


In [16]:
# def get_cleaned_data(input_data, mode='df'):
#     stop = stopwords.words('english')
    
#     input_df = ''
    
#     if mode != 'df':
#         input_df = pd.DataFrame([input_data], columns=['question1','question2'])
#     else:
#         input_df = input_data
        
#     #lowercase the text
#     input_df['question1'] = input_df['question1'].str.lower()
#     input_df['question2'] = input_df['question2'].str.lower()
    
#     input_df['question1'] = input_df['question1'].apply(lambda elem: decontracted(elem))
#     input_df['question2'] = input_df['question2'].apply(lambda elem: decontracted(elem))
    
#     #remove special characters
#     input_df['question1'] = input_df['question1'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
#     input_df['question2'] = input_df['question2'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    
#     # remove numbers
#     input_df['question1'] = input_df['question1'].apply(lambda elem: re.sub(r"\d+", "", elem))
#     input_df['question2'] = input_df['question2'].apply(lambda elem: re.sub(r"\d+", "", elem))
    
   
#     input_df['question1'] = input_df['question1'].apply(lambda words: (wordnet_lemmatizer.lemmatize(words)))
#     input_df['question2'] = input_df['question2'].apply(lambda words: (wordnet_lemmatizer.lemmatize(words)))
    
#     return input_df

In [17]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
199995,199995,394682,394683,Why are some answers collapsed?,Why do some answers get collapsed on Quora?,1
199996,199996,394684,394685,What can I do about my family being homophobic?,What do I do about my family being homophobic?,1
199997,199997,394686,394687,How can India be a secular century when there ...,Are there any downsides to implementation of u...,1
199998,199998,394688,394689,What is it like to date an extremely attractiv...,How do extremely attractive people affect the ...,0


# Making corpus to be made fit on the texts of both question 1 and question 2:-

In [18]:
features = ['question1','question2']

In [19]:
corpus = []

In [20]:
len(df['question1'])

200000

In [21]:
for f in features:
    corpus+=df[f].values.tolist()

In [22]:
# corpus

In [23]:
len(corpus)

400000

# Tokenizing and fitting the text:-

In [24]:
max_words=40000
max_seq_length=40
dim=300

# Tokenizer num_words: the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.

In [25]:
tokenizer=Tokenizer(num_words=40000)
tokenizer.fit_on_texts(corpus)

# Texts to sequences:-

In [26]:
X_q1 = tokenizer.texts_to_sequences(df['question1'])
X_q2 = tokenizer.texts_to_sequences(df['question2'])

In [27]:
X_test_q1 = tokenizer.texts_to_sequences(df_test['question1'])
X_test_q2 = tokenizer.texts_to_sequences(df_test['question2'])

In [28]:
X_q1 = pad_sequences(X_q1, maxlen=40)
X_q2 = pad_sequences(X_q2, maxlen=40)

In [29]:
X_test_q1 = pad_sequences(X_test_q1,maxlen=40)
X_test_q2 = pad_sequences(X_test_q2,maxlen=40)

In [30]:
Y=df['is_duplicate'].values
Y

array(['0', '0', '0', ..., '1', '0', '0'], dtype=object)

In [31]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 68803 unique tokens


# So we have found that we have 68803 unique tokens. So we will take round 60000 words as our embedding input words.

# For embedding matrix we will be using pretrained model word2vec:-
# Now importing the pretrained embedding index from Google index

In [32]:
path='/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin'

In [33]:
from gensim.models.keyedvectors import KeyedVectors

In [34]:
wv_from_bin = KeyedVectors.load_word2vec_format(path, binary=True, limit=500000) 
  #extracting word vectors from google news vector
embeddings_index = {}
for word, vector in zip(wv_from_bin.vocab, wv_from_bin.vectors):
      coefs = np.asarray(vector, dtype='float32')
      embeddings_index[word] = coefs

In [35]:
print('Found %s word vectors.' % len(embeddings_index))

Found 500000 word vectors.


# Word2vec have 500000 word vectors.

In [36]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

68804

# We have 68804 unique word vectors. So this will be our embedding matrix length.

In [37]:
len(word_index)

68803

In [38]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [39]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08007812,  0.10498047,  0.04980469, ...,  0.00366211,
         0.04760742, -0.06884766],
       [ 0.13964844, -0.00616455,  0.21484375, ...,  0.05712891,
         0.09960938, -0.234375  ],
       ...,
       [ 0.13183594,  0.40625   , -0.17480469, ...,  0.13378906,
        -0.19042969, -0.01904297],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

# As word2vec has 300 dimensions so we will be choosing output dimension of embedding layer as 300 units..

In [41]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Train Test and Split:-

In [44]:
X_train_q1,X_val_q1,X_train_q2,X_val_q2,y_train,y_val = train_test_split(X_q1,X_q2,Y,test_size=0.2,random_state=42)
X_train = [X_train_q1,X_train_q2]
X_val = [X_val_q1,X_val_q2]
X_test = [X_test_q1,X_test_q2]

(160000, 40) (40000, 40)


In [49]:
print(X_train_q1.shape,X_val_q1.shape)

(160000, 40) (40000, 40)


In [50]:
print(X_train_q2.shape,X_val_q2.shape)

(160000, 40) (40000, 40)


# X train is the combination of X_train_q1 and X_train_q2..

In [51]:
X_train

[array([[    0,     0,     0, ...,  1293, 13680,   485],
        [    0,     0,     0, ...,    41,     1, 11747],
        [    0,     0,     0, ...,     7, 22896,   228],
        ...,
        [    0,     0,     0, ...,   588,  4832,  9243],
        [  674,    59,    89, ...,    30,     5,     9],
        [    0,     0,     0, ...,   143,    15,    40]], dtype=int32),
 array([[    0,     0,     0, ..., 10464, 13680,   485],
        [    0,     0,     0, ...,   459,    10,    66],
        [    0,     0,     0, ...,     7, 22896,   228],
        ...,
        [    0,     0,     0, ...,   937,    10,    53],
        [    0,     0,     0, ...,   745,  4408,    20],
        [    0,     0,     0, ...,   199,     7,   299]], dtype=int32)]

# Siamese Network Architecture:-

In [53]:
input_q1=Input(shape=(40,), dtype='int32')
input_q2=Input(shape=(40,), dtype='int32')

In [57]:
if embedding_matrix is not None:
    weights=[embedding_matrix],

    embedding_layer = Embedding(vocab_size,
                300,
                input_length=40,
                weights=[embedding_matrix],
                trainable=False)
else:
    embedding_layer = Embedding(vocab_size,
                300,
                input_length=40,
                trainable=True)


In [60]:
embedded_sequences_q1=embedding_layer(input_q1)
embedded_sequences_q2=embedding_layer(input_q2)

In [61]:
embedded_sequences_q1

<tf.Tensor 'embedding_1_2/Identity:0' shape=(None, 40, 300) dtype=float32>

In [62]:
layer=Bidirectional(LSTM(units=200,return_sequences=False))

In [63]:
x1=layer(embedded_sequences_q1)
x2=layer(embedded_sequences_q2)

In [65]:
diff = subtract([x1,x2])
summation = add([x1,x2])
    
x = concatenate([x1,x2,diff,summation],1)

x = Dense(64,activation='relu')(x)
    
preds = Dense(1,activation='sigmoid')(x)


model = Model(inputs=[input_q1,input_q2],outputs=preds)
model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [ ]:
model.summary()